In [14]:
import numpy as np

In [3]:
#MSE
def mean_squared_error(y, t):
    return 0.5*np.sum((y-t)**2) 

In [5]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t*np.log(y+h))

In [8]:
# CEE for Batch
def cross_entropy_error_batch(y, t):
    
    if y.ndim == 1:
        y = y.reshape(1, y.size)
        t = t.reshape(1, t.size)
    
    delta = 1e-7
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y+delta)) / batch_size
   
    

In [9]:
# 수치 미분
def numerial_diff(f, x):
    h = 1e-4

    return (f(x+h)-f(x-h)) / (2*h)

In [12]:
# 편미분과 기울기
def numerical_gradeient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        
        x[idx] = tmp_val + h
        fxh1 = f(x)
    
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        it.iternext()

    return grad
    

In [13]:
#Gradient Descent
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradeient(f, x)
        x -= lr * grad
    return x

In [15]:
class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2, 3)

    def predict(self, x):
        return x @ self.W 

    def softmax(self, x):
        c = np.max(x)
        exp_x = np.exp(x-c)
        exp_all = np.sum(exp_x)
        
        return exp_x / exp_all
        
    

    def loss(self, x, t):
        z = self.predict(x)
        y = self.softmax(z)
        
        loss = cross_entropy_error(z,y)
        return loss

In [17]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        
    def sigmoid(self, x):
        return 1 + (1+np.exp(-x))
    
    def softmax(self, x):
        c = np.max(x)
        exp_x = np.exp(x-c)
        exp_all = np.sum(exp_x)
        
        return exp_x / exp_all
        
        
    def cross_entropy_error(self, y, t):
        
        if y.ndim == 1:
            y = y.reshape(1, y.size)
            t = t.reshape(1, t.size)
            
        batch_size = y.shape[0]
        delta = 1e-4
        return -np.sum(t*np.log(y+delta)) / batch_size
     
        
    
    def loss(self, x ,t):
        y = self.predict(x)
        
        return self.cross_entropy_error(y, t)
    
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = self.softmax(a1)
        a2 = np.dot(x, W1) + b1
        y = self.softmax(a2)
        
        return y
        
        
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def numerical_gradient(self, f, x):
        
        h = 1e-4
        grads = np.zeros_like(x)
        
        it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
        
        while not it.finished:
            idx = it.multi_index
            tmp_val = x[idx]
            
            x[idx] = tmp_val + h
            fxh1 = f(x)
            
            x[idx] = tmp_val - h
            fxh2 = f(x)
            
            grads[idx] = (fxh1 - fxh2) / (2*h)
            x[idx] = tmp_val
            it.iternext()
            
        return grads
        
    
    def numerical_gradient_all(self, x, t):
        
        loss_W = lambda W: self.loss(x, t)
        
        
        grads = {}
        grads['W1'] = self.numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = self.numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = self.numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = self.numerical_gradient(loss_W, self.params['b2'])
        
        return grad
            
            
            
        

In [ ]:
net = TwoLayerNet(input_size = 784, hidden_size = 100, output_size=10)
print(net.params['W1'].shape)  # (784, 100)
print(net.params['b1'].shape)  # (100,)
print(net.params['W2'].shape)  # (100, 10)
print(net.params['b2'].shape)  # (10,)

x = np.random.rand(100, 784)  # 더미 입력 데이터(100장 분량)
# 100 * 100
# (100 * 100) * (100 * 10) => (100, 10)
t = np.random.rand(100, 10)   # 더미 정답 레이블(100장 분량)

grads = net.numerical_gradient_all(x, t)  # 기울기 계산

(784, 100)
(100,)
(100, 10)
(10,)


In [ ]:
print(grads['W1'].shape)  # (784, 100)
print(grads['b1'].shape)  # (100,)
print(grads['W2'].shape)  # (100, 10)
print(grads['b2'].shape)  # (10,)

In [ ]:
from dataset03.dataset.mnist import load_mnist
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

In [ ]:
print(x_train.shape)
print(t_train.shape)
print(x_test.shape)
print(t_test.shape)

In [ ]:
iters_num = 1000 
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size/batch_size, 1)

In [19]:
from tqdm.auto import tqdm # progress bar

for i in tqdm(range(iters_num)):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grads = network.numerical_gradient_all(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1','b1','W2','b2'):
         network.params[key] -= learning_rate * grads[key]
        
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
     # 1에폭 당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | "+ str(train_acc) + ", " + str(test_acc))
    

NameError: name 'iters_num' is not defined